In [10]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def weighted_divisor_sum(k,r):
    ans=0
    pfk=prime_factors(k)
    weights=[]
    for d in divisors(k):
        weight=1
        pfd=prime_factors(d)
        for p in pfd:
            opd=order(p,d)
            opk=order(p,k)
            wt=binomial(opk,opd)
            weight=weight*wt
        weights=weights+[weight]
        ans=ans+weight*d^r
    return ans

def prime_factors(n):
    F=factor(n)
    f0=[f[0] for f in F]
    return sortAndSmash(f0)

def omega(n):
    if n==1:return 1
    if n>1:return len(prime_factors(n))

def omega2(n):
    return len(prime_factors(n))

def bigOmega(n):
    F=factor(n)
    ans=0
    for f in F:
        ans=ans+f[1]
    return ans

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [3]:
wtDvSrs=0
for n in [1..10]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,1)*x^n
print(wtDvSrs)
print(wtDvSrs.truncate(4))

18*x^10 + 16*x^9 + 27*x^8 + 8*x^7 + 12*x^6 + 6*x^5 + 9*x^4 + 4*x^3 + 3*x^2 + x
4*x^3 + 3*x^2 + x


In [4]:
data=[]
for n in [1..50]:
    w=weighted_divisor_sum(n,1)
    data=data+[w]
print(data) # A003959		If n = Product p(k)^e(k) then a(n) = Product (p(k)+1)^e(k), a(1) = 1.

[1, 3, 4, 9, 6, 12, 8, 27, 16, 18, 12, 36, 14, 24, 24, 81, 18, 48, 20, 54, 32, 36, 24, 108, 36, 42, 64, 72, 30, 72, 32, 243, 48, 54, 48, 144, 38, 60, 56, 162, 42, 96, 44, 108, 96, 72, 48, 324, 64, 108]


In [9]:
wtDvSrs=0
for n in [1..10000]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,1)*x^n
    
import time

def recip_wtDv(n):
    g=L(1/wtDvSrs.truncate(2*n))
    return g

def recip_wtDvPowerConstant(n):
    f=recip_wtDv(n+1)^n
    return f[0]


start=time.time()

    
L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run16may23no2.txt','wb') 
data=[]
start=time.time()/3600
for a in [1..1000]:
    c=recip_wtDvPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1, 2.1818268578499556e-05)
(2, 0.00038815801963210106)
(3, 0.0011974394437856972)
(4, 0.0019422274781391025)
(5, 0.0030471855425275862)
(6, 0.004160958051215857)
(7, 0.005674509156960994)
(8, 0.006724553008098155)
(9, 0.008144409977830946)
(10, 0.00957836938323453)
(11, 0.01138856157194823)
(12, 0.01282882917439565)
(13, 0.014632627426180989)
(14, 0.016449568327516317)
(15, 0.01865444303257391)
(16, 0.02005571307381615)
(17, 0.021816248015966266)
(18, 0.023602236644364893)
(19, 0.02576857915846631)
(20, 0.027530589955858886)
(21, 0.02968564018374309)
(22, 0.03183843690203503)
(23, 0.03442104801069945)
(24, 0.03619992022868246)
(25, 0.03835381555836648)
(26, 0.04051746806362644)
(27, 0.04305910662515089)
(28, 0.04523812688421458)
(29, 0.04777745861792937)
(30, 0.050328464130871)
(31, 0.053305653331335634)
(32, 0.05499865801539272)
(33, 0.05708229943411425)
(34, 0.05916336632799357)
(35, 0.061633917735889554)
(36, 0.06372935528634116)
(37, 0.06619107408914715)
(38, 0.06869335193186998)


(316, 1.0334058732842095)
(317, 1.0379925855086185)
(318, 1.0425697163445875)
(319, 1.0475500674219802)
(320, 1.050653278536629)
(321, 1.0541368438862264)
(322, 1.0576133877621032)
(323, 1.0614243539166637)
(324, 1.0649012963403948)
(325, 1.0687362638418563)
(326, 1.072573708544951)
(327, 1.0767828658572398)
(328, 1.0802482899744064)
(329, 1.084082309214864)
(330, 1.0879304811242037)
(331, 1.0921356119215488)
(332, 1.0959571583080105)
(333, 1.1001503177685663)
(334, 1.104363153630402)
(335, 1.1089200696442276)
(336, 1.1123967872117646)
(337, 1.1162280305870809)
(338, 1.120089002768509)
(339, 1.124303836375475)
(340, 1.128243396698963)
(341, 1.132453281315975)
(342, 1.1366723055252805)
(343, 1.1412561038159765)
(344, 1.145099873363506)
(345, 1.1492946157814004)
(346, 1.1535120130865835)
(347, 1.1580904514412396)
(348, 1.1623091194196604)
(349, 1.1668720961315557)
(350, 1.1714633352239616)
(351, 1.1764311369042844)
(352, 1.1799494049628265)
(353, 1.1837654822156765)
(354, 1.1875965955550

(636, 2.4381178675103)
(637, 2.443423800810706)
(638, 2.4487224165932275)
(639, 2.454430343292188)
(640, 2.457933817699086)
(641, 2.4617977546877228)
(642, 2.4656424018903635)
(643, 2.469864715007134)
(644, 2.4737362693995237)
(645, 2.477936721930746)
(646, 2.4821501244441606)
(647, 2.486733495257795)
(648, 2.4905787116731517)
(649, 2.494806250790134)
(650, 2.4990127625060268)
(651, 2.5035920058144256)
(652, 2.507829488313291)
(653, 2.5124196244287305)
(654, 2.5170174530358054)
(655, 2.521951508591883)
(656, 2.5257967588840984)
(657, 2.530014799966011)
(658, 2.5342312082648277)
(659, 2.538839388813358)
(660, 2.543092909443658)
(661, 2.5476706601912156)
(662, 2.552275355788879)
(663, 2.5572344722459093)
(664, 2.5615765913389623)
(665, 2.5662597732734866)
(666, 2.570847061695531)
(667, 2.5758016263716854)
(668, 2.5803953491849825)
(669, 2.585337882512249)
(670, 2.5902957949438132)
(671, 2.595614453894086)
(672, 2.5994545077555813)
(673, 2.603683421621099)
(674, 2.607891088584438)
(675, 2

(958, 4.022117083019111)
(959, 4.02840497775469)
(960, 4.032847843889613)
(961, 4.037684228271246)
(962, 4.042560877802316)
(963, 4.047785724746063)
(964, 4.052587641694117)
(965, 4.057779118069448)
(966, 4.063013375853188)
(967, 4.068522538582329)
(968, 4.073331400228199)
(969, 4.078514979162719)
(970, 4.083704263321124)
(971, 4.0892743724398315)
(972, 4.094454416655935)
(973, 4.100045966042671)
(974, 4.105597202433273)
(975, 4.111474551376887)
(976, 4.1162683852599)
(977, 4.121461250819266)
(978, 4.126697946048807)
(979, 4.132239693833981)
(980, 4.137426023604348)
(981, 4.142967758583836)
(982, 4.148537352215499)
(983, 4.154473574773874)
(984, 4.15967973304214)
(985, 4.165272353624459)
(986, 4.170844116655644)
(987, 4.176765385782346)
(988, 4.182315371348523)
(989, 4.18828423024388)
(990, 4.194228864100296)
(991, 4.200504278589506)
(992, 4.20533912855899)
(993, 4.210566217487212)
(994, 4.215761953033507)
(995, 4.221327679930255)
(996, 4.226514110807329)
(997, 4.23208059836179)
(998, 

In [2]:
import pickle
rfile=open('/Users/barrybrent/run16may23no2.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
print(len(s))

1000


In [4]:
import pickle
rfile=open('/Users/barrybrent/run16may23no2.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    if o2==0:yes=yes+1
    if o2!=0:no=no+1
print((yes,no))

(1000, 0)
